In [ ]:
# 引入第三方包
from pprint import pprint
import os,json
import pandas as pd
import pickle, json
from tqdm import tqdm,trange
import numpy as np
from matplotlib import pyplot as plt
from scipy import interpolate
import torch
import math
# 屏蔽warning
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# 数据合规性检查
from OmniScape_nav_lint import *
lint = OmniScapeNavLint()

# from msgRegister import *
# gravity_acceleration (float): 重力加速度的值，用于单位转换。
# degree_to_rad (float): 度到弧度的转换系数，用于角速度单位转换。

gravity_acceleration = 9.8066
degree_to_rad = math.pi/180

In [ ]:
# DataReader Utils
def llh2map(llh,p0):
    """
    将经纬高坐标(LLH)转换为局部地图坐标系
    
    参数:
    llh: 包含纬度(lat)、经度(lon)和高度(alt)的列表或数组
    p0: 参考点的经纬高坐标，作为局部坐标系的原点
    
    返回:
    result: 转换后的局部坐标，相对于参考点p0的偏移量(x,y,z)
    """
    scale = None  # 比例因子，用于经度方向的投影校正
    origin = None  # 原点坐标
    [lat0,lon0,alt0] = p0  # 解析参考点坐标
    [lat,lon,alt] = llh  # 解析目标点坐标
    if scale is None:
        # 计算比例因子，基于参考纬度的余弦值
        scale = np.cos(lat0 * np.pi / 180.)
    er = 6378137.  # 地球半径(米)
    if origin is None:
        # 计算参考点在墨卡托投影下的坐标
        tx0 = scale * lon0 * np.pi * er / 180.  # 参考点x坐标
        ty0 = scale * er * np.log(np.tan((90. + lat0) * np.pi / 360.))  # 参考点y坐标
        tz0 = alt0  # 参考点z坐标(高度)
        t0 = np.array([tx0, ty0, tz0])
        origin = t0  # 设置原点
        origin[-1] = 0
    # 计算目标点在墨卡托投影下的坐标
    tx = scale * lon * np.pi * er / 180.  # 目标点x坐标
    ty = scale * er * np.log(np.tan((90. + lat) * np.pi / 360.))  # 目标点y坐标
    tz = alt  # 目标点z坐标(高度)
    t = np.array([tx, ty, tz])
    result = t - origin  # 计算相对于原点的偏移量
    return result

def secNanosec2sec(sec,nanosec):
    return sec + nanosec/1e9


def timeseriesDataInterpolation(srcTS,srcData,tgtTS):
    # zero-dim插值：
    # 输入：原序列A时间戳，原序列A数据，目标序列B时间戳
    # 功能：对原序列A数据从原序列A时间戳向目标序列B时间戳进行zero-dim插值，得到目标序列B数据
    # 输出：目标序列B数据
    # filter tgtTs>min(srcTS) and tgtTs<max(srcTS)
    tgtTSinter = tgtTS[(tgtTS>=min(srcTS)) & (tgtTS<=max(srcTS))]
    f = interpolate.interp1d(srcTS, srcData, kind='zero')
    tgtData = f(tgtTSinter)
    # 对于tgtTSinter中不在srcTS中的数据，统计前：(tgtTS>=min(srcTS)) 个数，在tgtData前补齐
    # 对于tgtTSinter中不在srcTS中的数据，统计后：(tgtTS<=max(srcTS)) 个数，在tgtData后补齐
    # frontPadLength = 前：(tgtTS>=min(srcTS)) 个数
    # backPadLength = 后：(tgtTS<=max(srcTS)) 个数
    # print("前长度：",len(tgtData))
    frontPadLength = np.sum(tgtTS < min(srcTS))
    backPadLength = np.sum(tgtTS > max(srcTS))
    tgtData = np.pad(tgtData, (frontPadLength, backPadLength), mode='edge')
    # print("后长度：",len(tgtData),"补充：",frontPadLength,backPadLength)
    return tgtTSinter,tgtData

# def flagDataMatch(srcTS,srcData,tgtTS):# 以固定距离为阈值，对非插值flag置1
#     print("trange for 对非插值flag置1")
#     UTCIdxLast = 0
#     srcFlag = srcData.copy()
#     for imuIdx in trange(len(tgtTS)):
#         if imuIdx >= len(srcData): break
#         srcData[imuIdx] = 0
#         for UTCIdx in range(UTCIdxLast,len(srcTS)):
#             imuT,utcT = tgtTS[imuIdx],srcTS[UTCIdx]
#             # if abs(imuT-utcT)<1/10 and srcFlag[imuIdx] > 0:
#             if abs(imuT-utcT)<1/100/3 : # and srcFlag[imuIdx] > 0:
#                 srcData[imuIdx] = 1
#                 UTCIdxLast = UTCIdx
#                 # print(UTCIdxLast)
#                 break
#             if utcT-imuT > 1:
#                 # print(utcT-imuT)
#                 break
#     return srcData

def flagDataMatch(srcTS,srcData,tgtTS):
    print("trange for 对非插值flag置1")
    UTCIdxLast = 0
    srcFlag = srcData.copy()
    for srcTSItemIndex in trange(len(srcTS)):
        # 在tgtTS中找srcTSItem的最近值，给出srcTSItem的索引matchIndex，并把srcData[matchIndex]置1
        matchIndex = np.argmin(np.abs(tgtTS - srcTS[srcTSItemIndex]))
        srcData[matchIndex] = 1
        # print(srcTSItemIndex,srcTS[srcTSItemIndex],matchIndex)
    return srcData
def secNanosec2sec(sec,nanosec):
    return sec + nanosec/1e9
def imgFileName2timeStamp(imgFileName):
    return float(imgFileName.split(" ")[-1][:-4])
def timeseriesDataInterpolationStr(srcTS,srcData,tgtTS):
    """
    对源时间序列数据进行插值，生成目标时间序列对应的数据
    
    参数:
    srcTS: 源时间序列数组
    srcData: 源数据数组
    tgtTS: 目标时间序列数组
    
    返回:
    tgtTSinter: 插值后的目标时间序列
    tgtData: 插值后的目标数据
    flag: 标记数组,1表示该时刻有原始数据,0表示插值数据
    """
    # 截取目标时间序列在源时间序列范围内的部分
    tgtTSinter = tgtTS[(tgtTS>=min(srcTS)) & (tgtTS<=max(srcTS))]
    
    # 对每个目标时间点,找到最近的源数据点
    tgtData = []
    for tgtTSItem in tqdm(tgtTSinter):
        # 找到最近的源数据点索引
        closestIndex = np.argmin(np.abs(srcTS - tgtTSItem))
        tgtData.append(srcData[closestIndex])
        
    # 计算需要在前后补充的数据长度
    frontPadLength = np.sum(tgtTS < min(srcTS))  # 前面需要补充的长度
    backPadLength = np.sum(tgtTS > max(srcTS))   # 后面需要补充的长度
    # print("对齐长度前,",len(tgtData))
    # 在数据前后进行补充,使用首尾数据进行填充
    tgtData = [tgtData[0]]*frontPadLength+tgtData+[tgtData[-1]]*backPadLength
    
    # print("对齐时间序列长度",len(tgtTS))
    
    # print("对齐长度后,",len(tgtData))
    # 初始化标记数组,用于标记原始数据点
    flag = np.zeros(len(tgtTS))
    
    # 对每个源时间点,在目标时间序列中找到最近点并标记
    for srcTSItem in tqdm(srcTS):
        matchIndex = np.argmin(np.abs(tgtTS - srcTSItem))
        flag[matchIndex] = 1

    return tgtTSinter,tgtData,flag
    


In [ ]:
with open("name.ros_pickle","rb") as f:
    rosPickle = pickle.load(f)
rosPickle.keys()
for k,v in rosPickle.items():
    if isinstance(v, dict):
        print(f"{k}: 字典类型，包含 {len(v)} 个键值对")
    elif isinstance(v, list):
        print(f"{k}: 列表类型，长度为 {len(v)}")
    elif isinstance(v, str):
        print(f"{k}: 字符串类型，长度为 {len(v)}")
    elif isinstance(v, np.ndarray):
        print(f"{k}: numpy数组类型，形状为 {v.shape}")
    else:
        print(f"{k}: {type(v)}类型，值为 {v}")


*.omni_pickle是一个包含传感器数据和处理后数据的字典，用于导航和定位应用。以下是对各变量的详细分析：
| 变量名 | 维度 | 维度意义 | 单位及含义 |
|-------|------|---------|-----------|
| t | [N] | 时间序列数组 | 浮点数，秒(s)，表示数据点的时间戳，是实际时间戳与t0的差 |
| p_gt | [N, 3] | 位置真值数组，每行包含3个坐标值 | ENU (East-North-Up) 坐标系下的位置|
| ang_gt | [N, 3] | 角度真值数组，每行包含3个角度值 | 弧度(rad)，表示roll(横滚)、pitch(俯仰)、yaw(偏航)角 |
| v_gt | [N, 3] | 速度真值数组，每行包含3个速度分量 | 米/秒(m/s)，ENU坐标系下的速度分量 |
| u | [N, 6] | IMU测量数据，每行包含6个传感器值 | 前3列为角速度(rad/s)，后3列为加速度(m/s²) |
| name | 标量字符串 | 数据集名称 | 字符串，表示当前数据集 |
| t0 | [1] | 初始时间戳 | 秒(s)，表示数据序列的起始时间 |
| p0 | [3] | 初始位置 | 度(deg)，表示初始纬度、经度、高程坐标 |
| gnss | [N, 5] | GNSS测量数据 | 前3列为ENU坐标(m)，第4列为卫星数量，第5列为有效标志 |
| gnss_feature | list[dict] | GNSS特征数据 | 包含时间、卫星数量等信息的字典列表 |
| gnss_feature_tensor | [N, 15] | GNSS特征的张量表示 | 包含卫星信号强度、位置精度等多种特征 |
| img_path | list[dict{"sensor":"xxx","path":list[[path,flag],[path,flag],...]}] | GNSS特征的张量表示 | 包含卫星信号强度、位置精度等多种特征 |


详细说明：
1. 时间相关变量：
    - t：表示每个数据点的时间戳，按顺序记录整个轨迹
    - t0：初始时间点，作为参考时间
2. 位置与姿态变量：
    - p_gt：Ground Truth位置，在ENU坐标系下，表示东、北、上三个方向的坐标
    - ang_gt：Ground Truth角度，记录车辆或设备的姿态角
    - p0：初始位置的经纬度坐标和海拔高度
3. 运动学变量：
    - v_gt：Ground Truth速度，表示ENU坐标系下的速度分量
4. 传感器数据：
    - u：IMU传感器数据，包括陀螺仪测量的角速度和加速度计测量的线性加速度
    - gnss：GNSS接收机数据，包含位置和状态信息
    - gnss_feature与gnss_feature_tensor：GNSS的扩展特征数据，用于深度学习模型训练
      - 1、可见卫星数量 VSR_feature
      - 2、载噪比>30的可见卫星数量VSR_cn0_feature
      - 3、高度角<45的可见卫星数量VSR_ele_feature
      - 4、平均载噪比 mean_cn0
      - 5、载噪比的标准差std_cn0
      - 6、uq_载噪比(upper quartile 四分位数)up_qu_cn0
      - 7、lq_载噪比(lower quartile 四分位数)low_qu_cn0
      - 8、载噪比（elev<30）?cn0_elev<30
      - 9、mean_MP
      - 10、range_MP
      - 11、mean_MP（elev<60）
      - 12、平均高度角 mean_elev
      - 13、高度角标准差 std_elev
      - 14、GDOP
      - 15、VDOP

In [ ]:
# with open("../AI-IMU-DR-dataMatching/1_UrbanNav-HK-Medium-Urban.p","rb") as f:
#     demoPickle = pickle.load(f)
# demoPickle.keys()
# for k,v in demoPickle.items():
#     # print(type(v))
#     if isinstance(v, torch.Tensor):
#         print(k, v.shape)
#     else:
#         print(k, type(v))

In [ ]:
mondict = {}

In [ ]:
mondict["t0"] = secNanosec2sec(rosPickle['Base'][0]['stamp_sec'],rosPickle['Base'][0]['stamp_nanosec'])
mondict["t"] =  [secNanosec2sec(item['stamp_sec'],item['stamp_nanosec']) for item in rosPickle['Base']]
mondict["ang_gt"] =  [[item['fpd_gnssroll'],item['fpd_gnsspitch'],item['fpd_gnssheading']] for item in rosPickle['Base']]
mondict["llh_gt"] =  [[item['fpd_gnsslatitude'],item['fpd_gnsslongitude'],item['fpd_gnssaltitude']] for item in rosPickle['Base']]
mondict["v_gt"] =  [[item['fpd_gnssve'],item['fpd_gnssvn'],item['fpd_gnssvu']] for item in rosPickle['Base']]
mondict["u"] =  [[  item['angular_velocity_x'],item['angular_velocity_y'],item['angular_velocity_z'],item['linear_acceleration_x'],
                    item['linear_acceleration_y'],item['linear_acceleration_z']] for item in rosPickle['IMU']]
mondict["p0"] = mondict["llh_gt"][0]
mondict["u_gt"] = [[item['gyro_x'],item['gyro_y'],item['gyro_z'],item['acc_x'],
                    item['acc_y'],item['acc_z']] for item in rosPickle['Base']]
mondict["p_gt"] = []
for i in tqdm(range(len(mondict["llh_gt"]))):    mondict["p_gt"].append(llh2map(mondict["llh_gt"][i],mondict["p0"]))


In [ ]:
GNSSdataListLLH = []
for item in rosPickle['GNSS']:
    # print(item)
    lat = item['datahpposllh_lat']*1e-7+item['datahpposllh_lathp']*1e-9
    lon = item['datahpposllh_lon']*1e-7+item['datahpposllh_lonhp']*1e-9
    alt = item['datahpposllh_hmsl']*1e-3+item['datahpposllh_hmslhp']*1e-4
    enu = llh2map([lat,lon,alt],mondict["p0"])
    GNSSdataListLLH.append(
        {
            "time":secNanosec2sec(item['stamp_sec'],item['stamp_nanosec']),
            "lat":lat,            "lon":lon,            "alt":alt,
            "E":enu[0],            "N":enu[1],            "U":enu[2],
            "numOfSat":len(item['datasat_numsvs']) if type(item['datasat_numsvs'])==list else item['datasat_numsvs'],
            "Flag":0.0
        }
    )

### GNSS时间轴对齐(插值并标记flag)

In [ ]:
srcTime = np.array([item["time"] for item in GNSSdataListLLH])
tgtTime = np.array(mondict["t"])
print('gnss src time,',srcTime)
print('gnss tgt time,',tgtTime)
interpolationGNSSData = {}
for key in ["E","N","U","numOfSat","Flag"]:
    keyData = np.array([item[key] for item in GNSSdataListLLH])
    # print(keyData.shape)
    _,interpolationGNSSData[key]=timeseriesDataInterpolation(srcTime,keyData,tgtTime)
    
keyData = interpolationGNSSData["Flag"]
interpolationGNSSData["Flag"]=flagDataMatch(srcTime,keyData,tgtTime)

In [ ]:
interpolationGNSSData["Flag"][interpolationGNSSData["Flag"]>0]

In [ ]:
interpolationGNSSData_list = [
    {"E": e, "N": n, "U": u, "numOfSat": sat, "Flag": flag}
    for e, n, u, sat, flag in zip(
         interpolationGNSSData["E"],
         interpolationGNSSData["N"],
         interpolationGNSSData["U"],
         interpolationGNSSData["numOfSat"],
         interpolationGNSSData["Flag"],
    )
]

In [ ]:
mondict["gnss"] = [[item["E"],item["N"],item["U"],item["numOfSat"],item["Flag"]] for item in interpolationGNSSData_list]

### GNSSfeature


In [ ]:
gnss_feature_path = ".\\GNSSFeature\\nav_db.json"
def GNSSfeatureReader(path):
    with open(path,'r') as f:
        srcData = json.load(f)
    gnssFeatureData = []
    for k,v in srcData['_default'].items():
        if "dict_gnss_feature" in v:
            gnssFeatureDataItem = v['dict_gnss_feature']
            gnssFeatureDataItem['time'] = v["UTCTime"]
            gnssFeatureData.append(gnssFeatureDataItem)
    return gnssFeatureData
gnssFeatureData = GNSSfeatureReader(gnss_feature_path)

### GNSS feature时间轴对齐(插值并标记flag)

In [ ]:
gnssFeatureData[:2]

In [ ]:
keyList = []
for item in gnssFeatureData:    
    for k,v in item.items():        
        if k not in keyList:   keyList.append(k)
if "time" in keyList: keyList.remove("time")

In [ ]:
srcTime = np.array([item["time"] for item in gnssFeatureData])
tgtTime = np.array(mondict["t"])
print('gnss fea src time,',srcTime)
print('gnss fea tgt time,',tgtTime)
interpolationGNSSfeatureData = {}
for key in keyList:
    keyData = np.array([item[key] for item in gnssFeatureData])
    _,interpolationGNSSfeatureData[key]=timeseriesDataInterpolation(srcTime,keyData,tgtTime)

### 图片路径保存进*.p

In [ ]:
rosPickle['rosbagFilePath']

In [ ]:
rosPickle['toImgPath']

In [ ]:
mondict['imgPath'] = []
for imgPathItem in rosPickle['toImgPath']:
    imgGroupDict = {}
    imgGroupDict['sensor'] = imgPathItem['topic']
    imgGroupDict['basePath']  =  os.path.join(rosPickle['rosbagFilePath'],"toImg",imgPathItem['imgDir'])
    imgGroupDict['path'] = []
    for imgFileName in os.listdir(imgGroupDict['basePath']):
        imgGroupDict['path'].append([os.path.join(imgGroupDict['basePath'],imgFileName),imgFileName2timeStamp(imgFileName),1])
    srcTime = np.array([item[1] for item in imgGroupDict['path']])
    tgtTime = np.array(mondict["t"])
    print(len(srcTime),len([item[0] for item in imgGroupDict['path']]),len(tgtTime))
    print(srcTime[0],srcTime[-1],tgtTime[0],tgtTime[-1])
    _,tgtData,flag = timeseriesDataInterpolationStr(srcTime,[item[0] for item in imgGroupDict['path']],tgtTime)
    imgGroupDict['path'] = []
    for i in range(len(tgtData)):
        imgGroupDict['path'].append([tgtData[i],flag[i]])
    mondict['imgPath'].append(imgGroupDict)


In [ ]:
mondict["t"] =  [secNanosec2sec(item['stamp_sec'],item['stamp_nanosec'])-mondict["t0"] for item in rosPickle['Base']]

In [ ]:
for k,v in interpolationGNSSfeatureData.items():
    print(k,len(v))

In [ ]:

feature_keys = [
    'VSR_feature',     'VSR_cn0_feature',     'VSR_ele_feature', 
    'mean_cn0',     'std_cn0',     'up_qu_cn0',     'low_qu_cn0',     'cn0_elev', 
    'mean_MP',     'range_MP',     'mean_MP_elev',     'mean_elev',     'std_elev', 
    'GDOP',     'VDOP'
]

gnss_features = []
for i in range(len(interpolationGNSSfeatureData[feature_keys[0]])):
    feature_vector = []
    for key in feature_keys:
        feature_vector.append(interpolationGNSSfeatureData[key][i])
    gnss_features.append(feature_vector)

mondict["gnss_feature_tensor"] = np.array(gnss_features)

### gnss_feature

In [ ]:
sum([item[-1] for item in mondict["gnss"]])

In [ ]:
mondict["gnss_feature_tensor"]=interpolationGNSSfeatureData

In [ ]:
for k,v in mondict.items():
    print(k,type(v))
    if type(v) == list:
        mondict[k] = np.array(v)
# if "llh_gt" in mondict: del mondict['llh_gt']

In [ ]:
for k,v in mondict.items():
    print(k,type(v))
    try:
        print(k,v.shape)
    except AttributeError:
        try:
            print(k, len(v))
        except TypeError:
            print(k,v)

### gt单位与数据处理

In [ ]:
mondict['u_gt'][:,0:3] = mondict['u_gt'][:,0:3]*degree_to_rad
mondict['u_gt'][:,3:6] = mondict['u_gt'][:,3:6]*gravity_acceleration

In [ ]:
mondict['ang_gt'] = mondict['ang_gt'] * degree_to_rad

In [ ]:
mondict['ang_gt'][:,2] = mondict['ang_gt'][:,2]*(-1) + 2*math.pi

In [ ]:
mondict['llh_gt'][:,0] = mondict['llh_gt'][:,0] * degree_to_rad
mondict['llh_gt'][:,1] = mondict['llh_gt'][:,1] * degree_to_rad

### np.array 转 torch.tensor

In [ ]:
for k, v in mondict.items():
    print(k, type(v), getattr(v, "dtype", None))  # also check dtype
    if isinstance(v, np.ndarray):
        # if it's object dtype, try to coerce to float
        if v.dtype == np.object_:
            try:
                v = v.astype(np.float32)  # or np.float64
            except Exception as e:
                print(f"Could not cast {k}: {e}")
                continue
        mondict[k] = torch.tensor(v)

In [ ]:
with open("HIT_nav_pickle_0905kchtg.p","wb") as f:
    pickle.dump(mondict,f)